In [ ]:
import pandas as pd
import dask.dataframe as dd
from distributed import Client
from dask_kubernetes import KubeCluster, make_pod_spec

In [ ]:
pod_spec = make_pod_spec(
    image='docker-apac:6081/stem-pres-nyc-cab:latest',
    memory_request='10G',
    cpu_request=1,
    env={}
)

In [3]:
cluster = KubeCluster(pod_spec, namespace='stem-pres')
cluster.scale(100)
cluster

KeyboardInterrupt: 

In [4]:
client = Client(cluster.scheduler_address)
client

Client Scheduler: tcp://10.70.35.33:50404 Dashboard: http://10.70.35.33:8787/status,Cluster Workers: 53 Cores: 53 Memory: 512.00 GB


In [ ]:
columns = [
    'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
    'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 
    'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type',
    'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'
]

df = dd.read_csv(
    urlpath='s3://stem-pres/nyc_cab/yellow*2017*.csv.gz', 
    storage_options={
        'key': 'opt-minio-dev', 
        'secret': 'opt-minio-dev', 
        'client_kwargs': dict(endpoint_url='http://minio.opt-pilot.svc.hkcc.ks:9000/')
    },
    compression='gzip',
    blocksize=None,
    names=columns,
    header=0
)

df

In [6]:
df.persist()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
npartitions=175,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
data = _

In [7]:
df.get_partition(0).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
NaN,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0.0,CMT,2013-01-01 15:11:48,2013-01-01 15:18:10,4,1.0,-73.978165,40.757977,1,N,-73.98984,40.751173,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1.0,CMT,2013-01-06 00:18:35,2013-01-06 00:22:54,1,1.5,-74.00668,40.731781,1,N,-73.994499,40.750659,CSH,6.0,0.5,0.5,0.0,0.0,7.0
2.0,CMT,2013-01-05 18:49:41,2013-01-05 18:54:23,1,1.1,-74.004711,40.73777,1,N,-74.009831,40.726,CSH,5.5,1.0,0.5,0.0,0.0,7.0
3.0,CMT,2013-01-07 23:54:15,2013-01-07 23:58:20,2,0.7,-73.97459999999998,40.759945,1,N,-73.98473699999998,40.759388,CSH,5.0,0.5,0.5,0.0,0.0,6.0


In [10]:
data.vendor_id.value_counts()

CMT    2567250
VTS    2432750
Name: vendor_id, dtype: int64

In [11]:
from folium import Map
from folium.plugins import HeatMap

In [18]:
nyc_map = Map(location=[40.759945, -73.9745], zoom_start=12)
nyc_map

In [ ]:
data.pickup_latitude

In [25]:
hm_wide = HeatMap(
    zip(data.pickup_latitude.values, data.pickup_longitude.values, data.tip_amount.astype(float)), 
    min_opacity=0.2,
    max_val=data.tip_amount.max(),
    radius=17, blur=15, 
    max_zoom=1, 
)


hmap.add_to(nyc_map)

ValueError: could not convert string to float: 'tip_amount'

In [20]:
from geopandas import GeoDataFrame

In [21]:
gdf = GeoDataFrame(data)

In [22]:
gdf

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
NaN,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0.0,CMT,2013-01-01 15:11:48,2013-01-01 15:18:10,4,1.0,-73.978165,40.757977,1,N,-73.98984,40.751173,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1.0,CMT,2013-01-06 00:18:35,2013-01-06 00:22:54,1,1.5,-74.00668,40.731781,1,N,-73.994499,40.750659,CSH,6.0,0.5,0.5,0.0,0.0,7.0
2.0,CMT,2013-01-05 18:49:41,2013-01-05 18:54:23,1,1.1,-74.004711,40.73777,1,N,-74.009831,40.726,CSH,5.5,1.0,0.5,0.0,0.0,7.0
3.0,CMT,2013-01-07 23:54:15,2013-01-07 23:58:20,2,0.7,-73.97459999999998,40.759945,1,N,-73.98473699999998,40.759388,CSH,5.0,0.5,0.5,0.0,0.0,6.0
